### 🧠 MLflow Cheat Sheet — CRUD Operations (with Explanations)

This cheat sheet organizes MLflow by **CRUD operations**:
- **Create** → log & register
- **Read** → query & load
- **Update** → metadata & lifecycle
- **Delete** → cleanup

#### **CREATE (C)** — Create Experiments, Runs, Logs, Models


**Create an Experiment**

-  Creates an experiment if it does not exist
-  All future runs will be logged under this experiment
mlflow.set_experiment("Customer_Churn")


##### Create a Run
- A run represents a single execution of training or evaluation
with mlflow.start_run():
    pass

##### Log Parameters (Hyperparameters)
- Parameters are key-value pairs (immutable once logged)
mlflow.log_param("learning_rate", 0.01)
mlflow.log_param("batch_size", 32)


##### Log Metrics (Performance Values)
- Metrics track model performance
mlflow.log_metric("accuracy", 0.92)

- Metrics can be logged over time (e.g., epochs)
mlflow.log_metric("loss", 0.15, step=1)


##### Log Artifacts (Files & Outputs)
- Artifacts can be plots, models, logs, CSVs, etc.
mlflow.log_artifact("confusion_matrix.png")

- Log an entire directory
mlflow.log_artifacts("outputs/")


##### Log a Model
import mlflow.sklearn

- Saves the model along with dependencies
mlflow.sklearn.log_model(model, artifact_path="model")


##### Registers the logged model into the Model Registry
mlflow.register_model(
    "runs:/<run_id>/model",
    "ChurnModel"
)

#### READ (R) — View, Query, Load, Compare

- View Experiments & Runs (UI)
Launch MLflow UI
mlflow ui

- Open in browser:
http://localhost:5000
 

- Read Experiments Programmatically

from mlflow.tracking import MlflowClient

client = MlflowClient()

List all experiments
client.list_experiments()


- Read Runs with Filters
Search runs with metric-based filtering
client.search_runs(
    experiment_ids=["1"],
    filter_string="metrics.accuracy > 0.9"
)

- Read Parameters & Metrics of a Run
run = client.get_run(run_id)

Access logged parameters
run.data.params

Access logged metrics
run.data.metrics

- Load a Model for Inference
import mlflow.pyfunc

Load production-ready model
model = mlflow.pyfunc.load_model(
    "models:/ChurnModel/Production"
)

Run predictions
predictions = model.predict(data)

- Read Model Versions
Fetch all versions of a registered model
client.search_model_versions(
    "name='ChurnModel'"
)

#### UPDATE (U) — Modify Metadata & Lifecycle
MLflow does NOT allow updating:
Parameters
Metrics
Only metadata can be updated.

- Update Run Tags
Tags are editable metadata
mlflow.set_tag("dataset", "v2.1")
mlflow.set_tag("owner", "ml-team")

- Update Experiment Tags
Useful for organizing experiments
client.set_experiment_tag(
    experiment_id="1",
    key="project",
    value="Customer Retention"
)

- Update Model Stage (Lifecycle)
Promote model to Production
client.transition_model_version_stage(
    name="ChurnModel",
    version=3,
    stage="Production"
)

- Update Model Description
Add documentation to model versions
client.update_model_version(
    name="ChurnModel",
    version=3,
    description="XGBoost model trained on February dataset"
)

#### DELETE (D) — Cleanup (Soft Delete)
MLflow uses soft deletes (recoverable).

- Delete a Run
Marks run as deleted
client.delete_run(run_id)

- Delete an Experiment
Deletes experiment and all runs under it
client.delete_experiment(experiment_id)

- Delete a Registered Model
Removes model and all versions
client.delete_registered_model("ChurnModel")

- Delete a Model Version
client.delete_model_version(
    name="ChurnModel",
    version=3
)

- Restore Deleted Objects
Restore a deleted run
client.restore_run(run_id)

Restore a deleted experiment
client.restore_experiment(experiment_id)




### code 

**resources video to refer**

- https://www.youtube.com/watch?v=6ngxBkx05Fs

- https://www.youtube.com/watch?v=X2sx1lAsulQ

- https://www.youtube.com/watch?v=p268d7JjSZY

In [2]:
import pandas as pd
from typing import Dict, List, Text

In [3]:
# --- Linear Models ---
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso

# --- Tree Models ---
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# --- Ensemble Models ---
from sklearn.ensemble import (
    RandomForestClassifier, 
    RandomForestRegressor, 
    GradientBoostingRegressor, 
    AdaBoostClassifier,
    HistGradientBoostingClassifier  # Sklearn's version of LightGBM
)

# --- Support Vector Machines ---
from sklearn.svm import SVC, SVR

# --- Neighbors and Naive Bayes ---
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# --- Unsupervised Models ---
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

# --- Splitting Tool ---
from sklearn.model_selection import train_test_split

# --- Regression Metrics ---
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Dataset ---
from sklearn.datasets import fetch_california_housing

In [4]:
# fetch_california_housing downloads the data if you don't have it
housing = fetch_california_housing()

x = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.DataFrame(housing.target, columns=['MedHouseVal'])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8,random_state=42)
x_train.shape

(16512, 8)

##### linear regression

In [14]:
lr_model = LinearRegression()
lr_model.fit(x_train,y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [15]:
lr_pred= lr_model.predict(x_test)

lr_mse = mean_squared_error(y_test,lr_pred)
print(lr_mse)

lr_r2 = r2_score(y_test,lr_pred)
print(lr_r2)

0.5558915986952435
0.5757877060324514


##### gradient boosting

In [8]:
gb_model = GradientBoostingRegressor(
    learning_rate=0.1,
    n_estimators=150,
    criterion='squared_error',
    max_depth=5,
    min_samples_leaf=2
).fit(x_train,y_train)

gb_pred = gb_model.predict(x_test)

c:\sai files\python practice\fastapi\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [9]:
gb_mse= mean_squared_error(y_test,gb_pred)
print(gb_mse)
gb_r2= r2_score(y_test,gb_pred)
print(gb_r2)

0.2298462421652293
0.8245996128065483


### mlflow practice

In [16]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import mlflow.data
import mlflow.artifacts

##### manual logs --> log_params, log_metrics, log_model

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house price prediction gb")

# mlflow.autolog()
def mlflow_lr_pipeline(n_estimators,max_depth):

    with mlflow.start_run(run_name="gradient boosting model") as run:
        
        gb_model = GradientBoostingRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth
        )
        gb_model.fit(x_train,y_train.values.ravel())

        gb_pred = gb_model.predict(x_test)
        bg_mse = mean_squared_error(y_test,gb_pred)
        bg_r2= r2_score(y_test,gb_pred)
        
        # logging hyper-parameters to mlflow
        mlflow.log_param("n_estimators",n_estimators)
        mlflow.log_param("max_depth",max_depth)

        # logging evaluation metrics to mlflow
        mlflow.log_metric("bg_mse",bg_mse)        
        mlflow.log_metric("bg_r2",bg_r2)

        # storing code as artficat
        mlflow.log_artifact("mlflow.ipynb", artifact_path="mlflow.ipynb")

        # 'artifact_path' is the folder name inside MLflow where the model is saved
        mlflow.sklearn.log_model(sk_model=gb_model,
                                 name = "model_artifact")

        print(f"finsihed ml flow")
        return run.info.run_id


gb_run_id = mlflow_lr_pipeline(n_estimators=160,max_depth=5)


model_name = "gradient boosting model--"
model_uri = f"runs:/{gb_run_id}/model_artifact"

result = mlflow.register_model(model_uri, model_name)

print(f"Successfully registered model version: {result.version}")

Registered model 'gradient boosting model--' already exists. Creating a new version of this model...
2026/01/22 00:25:43 WARNING mlflow.tracking._model_registry.fluent: Run with id e052f4730c5548639cbdff9a14a2d0e6 has no artifacts at artifact path 'model_artifact', registering model based on models:/m-d38e6ab0abcd46188ddf316c85216dd2 instead


finsihed ml flow
Successfully registered model version: 6


Created version '6' of model 'gradient boosting model--'.


##### using auto_logs 

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house price prediction lr")

mlflow.autolog()

def mlflow_lr_pipeline(n_jobs):

    with mlflow.start_run(run_name="LinearRegression model") as run:
        
        lr_model = LinearRegression(n_jobs=n_jobs)
        lr_model.fit(x,y)

        lr_pred= lr_model.predict(x_test)

        lr_mse = mean_squared_error(y_test,lr_pred)
        print(lr_mse)

        lr_r2 = r2_score(y_test,lr_pred)
        print(lr_r2)

        mlflow.sklearn.log_model(sk_model=lr_model, name="lr_model_")

        return run.info.run_id
    
lr_run_id = mlflow_lr_pipeline(n_jobs=1)

model_name = "linear regression model--"
model_uri = f"runs:/{lr_run_id}/lr_model_"

result = mlflow.register_model(model_uri, model_name)

print(f"Successfully registered model version: {result.version}")


2026/01/22 00:41:26 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


0.5460476751617075
0.583299806221776


##### mlflow with optuna (parent child relation of verioning)

In [22]:
# Setup
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house price prediction gb - nested")

# 1. Enable Autolog (this replaces manual param, metric, and model logging)
mlflow.autolog(log_models=True)

def mlflow_gb_pipeline(n_estimators_list, max_depth):
    # 2. Start Parent Run
    with mlflow.start_run(run_name="GB_Parent_Search") as parent_run:
        
        for n_estimators in n_estimators_list:
            # 3. Start Child Run using nested=True
            with mlflow.start_run(run_name=f"est_{n_estimators}_depth_{max_depth}", nested=True):
                
                gb_model = GradientBoostingRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth
                )
                
                # Autolog captures everything during .fit()
                gb_model.fit(x_train, y_train.values.ravel())
                
        print("Finished MLflow Nested Runs")
        return parent_run.info.run_id

# Execute with multiple iterations
estimators = [100, 150, 200]
gb_parent_id = mlflow_gb_pipeline(n_estimators_list=estimators, max_depth=5)

2026/01/22 00:53:55 INFO mlflow.tracking.fluent: Experiment with name 'house price prediction gb - nested' does not exist. Creating a new experiment.
2026/01/22 00:53:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Finished MLflow Nested Runs


In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Setup
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house price prediction gb - random search")

# 1. Define distributions instead of fixed lists
param_dist = {
    "n_estimators": randint(50, 500),
    "max_depth": randint(3, 15),
    "learning_rate": uniform(0.01, 0.3),
    "subsample": uniform(0.6, 0.4) # sample between 0.6 and 1.0
}

rand_search = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(),
    param_distributions=param_dist,
    n_iter=10,  # Try 10 random combinations
    cv=3,
    n_jobs=-1
)

with mlflow.start_run(run_name="GB_Random_Search_Parent"):
    rand_search.fit(x_train, y_train.values.ravel())

2026/01/22 01:10:27 INFO mlflow.tracking.fluent: Experiment with name 'house price prediction gb - random search' does not exist. Creating a new experiment.
2026/01/22 01:13:55 INFO mlflow.sklearn.utils: Logging the 5 best runs, 5 runs will be omitted.


##### predict the model using model file

In [ ]:
# 1. Set the same tracking URI you used for training
mlflow.set_tracking_uri("http://localhost:5000")

# 2. Define the Model URI
# Format: models:/<model_name>/<version_or_stage>
model_name = "gradient_boosting_model_v1"
model_version = 1 
model_uri = f"models:/{model_name}/{model_version}"

# 3. Load the model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(model_uri)

# 4. Make Predictions
# Note: Ensure 'new_data' is a DataFrame or Numpy array with the same columns as x_test
# new_data = pd.DataFrame(...) 
predictions = loaded_model.predict(x_test)
print(predictions)

In [ ]:
# This creates a local web server (REST API) for your model
mlflow models serve -m "runs:/<YOUR_RUN_ID>/model" --port 5001

In [ ]:
# MLflow builds the Docker image for you!
mlflow models build-docker -m "runs:/<YOUR_RUN_ID>/model" -n "my-gradient-boost-app"